# HW 1: Corporate Bond Pricing (due by 9.20 Fri)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [17]:
import numpy as np

In [18]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [19]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.029960406534599, 10.0)

In [20]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=1e4):
    #generate exponential random numbers
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    
    #plug in random numbers and judge if default_time>mat 
    result=[]
    for time in default_time:
        if time<mat:
            result.append(recovery*np.exp(-rf_rate*time))
        else:
            result.append(np.exp(-rf_rate*mat))
    
    bond_price=np.average(np.array([result]))

    return bond_price






In [21]:
# Call your function
corp_bond(mat, def_rate, rf_rate, recovery, n_sample)

0.44153015525131106

In [22]:
# Find the mean and std by calling the function 100 times. 
data=[]
for i in range(100):
    data.append(corp_bond(mat, def_rate, rf_rate, recovery, n_sample))

np.mean(data),np.std(data)

(0.4406528262021967, 0.0021599341004380815)

### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [23]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [28]:
# Now include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=10, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    #first generate random number as usual
    U = np.random.uniform(size=int(n_sample))
    default_time=None
    
    if(antithetic):
        #perform the antithetic method
        U = np.random.uniform(size=int(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        

    if(mean_match):
        #generate random series and perform mean_match
        if antithetic:
            #if antithetic before, simply add the mean match
            default_time += 1/def_rate-default_time.mean()
        else:
            #if not antithetic, default_time is None need to redefine 
            default_time = -(1/def_rate)*np.log(U)
            default_time += 1/def_rate-default_time.mean()
            
    if default_time is None:
        #neither two methods are selected
        return None
    else:
        result1=[]
        for time in default_time:
            if time<mat:
                result1.append(recovery*np.exp(-rf_rate*time))
            else:
                result1.append(np.exp(-rf_rate*mat))
        
        return np.mean(result1)




In [34]:
# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
#antithetic
ant_res=[]
for i in range(100):
    price_item=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,True,False)
    ant_res.append(price_item)

print('antithetic method:mean:{},std:{}'.format(np.mean(ant_res),np.std(ant_res)))




antithetic method:mean:0.44006783530654786,std:0.0016236014931317825


In [35]:
#mean_match
mean_match_res=[]
for i in range(100):
    price_item=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,False,True)
    mean_match_res.append(price_item)

print('mean match method:mean:{},std:{}'.format(np.mean(mean_match_res),np.std(mean_match_res)))

mean match method:mean:0.4402906052591486,std:0.0014832739434590487


In [36]:
#integrated antithetic and mean_match
int_res=[]
for i in range(100):
    price_item=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,True,True)
    int_res.append(price_item)

print('mean match method:mean:{},std:{}'.format(np.mean(int_res),np.std(int_res)))

mean match method:mean:0.4405031488620215,std:0.0014859350246284552


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?d

ANS: Analytic solutions: $\int_0^T R\lambda e^{-\lambda t} e^{-rf t}dt+(1-(1-e^{-(rf+\lambda)t}))e^{-rf T}=\frac{\lambda R}{\lambda +R}(1-e^{-(rf+\lambda) T})+e^{-(\lambda +rf)T}$

In [37]:
### Put the analytic expression for the corporate bond price
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10
def analytic_solution(mat, def_rate, rf_rate, recovery):
    solution=def_rate*recovery/(def_rate+rf_rate)*(1-np.exp(-(def_rate+rf_rate)*mat))+np.exp(-(rf_rate+def_rate)*mat)
    return solution

In [38]:
analytic_solution(mat, def_rate, rf_rate, recovery)

0.44040907156462505